In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
#import optuna.integration.lightgbm as lgb
from datetime import date,timedelta

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
whole = pd.read_csv('whole.csv')
whole = whole.drop(['Unnamed: 0'], axis=1)

/home/yumezu/anaconda3/envs/kaggle/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
whole['date'] = pd.to_datetime(whole['date'].tolist())

In [4]:
ls = list(whole[whole['predict'] == 1]['date'].unique())

s = ls[0]

train = whole[whole['date'] < s]
train = train.append(train[train['date'] == s][train['hour'] == 0])
test = whole[whole['date'] == s][whole['predict'] == 1]

x = train.drop(['id', 'year', 'predict', 'date', 'events', 'bikes_available'], axis=1)
y = train['bikes_available']

tss = TimeSeriesSplit(n_splits=100)

for tr_idx, va_idx in tss.split(train):
    tr_x, va_x = x.iloc[tr_idx], x.iloc[va_idx]
    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]

lgb_train = lgb.Dataset(tr_x,tr_y)
lgb_eval = lgb.Dataset(va_x,va_y)

params = {
    'objective':'regression',
    'metric':'rmse',
}

model = lgb.train(params,lgb_train,valid_sets=[lgb_train,lgb_eval],
                early_stopping_rounds=100,
                verbose_eval=100)

model.params

{'objective': 'regression',
                'metric': 'rmse',
                'feature_pre_filter': False,
                'lambda_l1': 3.0219157234596604e-06,
                'lambda_l2': 2.6309832619595715e-05,
                'num_leaves': 137,
                'feature_fraction': 0.8,
                'bagging_fraction': 1.0,
                'bagging_freq': 0,
                'min_child_samples': 5,
                'num_iterations': 1000,
                'early_stopping_round': 100}

In [5]:
for i in range(len(ls)):
    
    if i == 0:
        print('############################################')
        print(str(i+1) + '/' + str(len(ls)))
        print('############################################')

        s = ls[i]

        train = whole[whole['date'] < s]
        train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
        test = whole[whole['date'] == s][whole['predict'] == 1]
        
        nan_ids = train[train['bikes_available'].isnull()]['id'].to_list()
        nan_ids[len(nan_ids):len(nan_ids)] = test[test['bikes_available'].isnull()]['id'].to_list()

        for i in range(len(nan_ids)):
            week_ago = whole[whole['id'] == nan_ids[i]]['date'] - timedelta(days=7)
            week_ago = week_ago.values[0]

            if (whole[whole['id'] == nan_ids[i]]['holiday'] == whole[whole['date'] == week_ago].iloc[0,9]).values[0]:
                week_ago = week_ago
            else:
                week_ago = whole[whole['id'] == nan_ids[i]]['date'] - timedelta(days=14)
                week_ago = week_ago.values[0]

            station_id = whole[whole['id'] == nan_ids[i]]['station_id']
            station_id = station_id.values[0]
            hour = whole[whole['id'] == nan_ids[i]]['hour']
            hour = hour.values[0]

            whole.loc[(whole['id'] == nan_ids[i]), 'bikes_available'] = whole[(whole['date'] == week_ago) & (whole['hour'] == hour) & (whole['station_id'] == station_id)]['bikes_available'].values[0]

        train = whole[whole['date'] < s]
        #train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
        test = whole[whole['date'] == s][whole['predict'] == 1]

        print('NaN_train ' + str(train['bikes_available'].isnull().sum()))
        print('NaN_test ' + str(test['bikes_available'].isnull().sum()))

        tss = TimeSeriesSplit(n_splits=2, test_size=744)

        for i in range(70):
            
            if i == 0:
                df = train[train['station_id'] == i]
        
                x = df.drop(['id', 'predict', 'date', 'events', 'bikes_available'], axis=1)
                y = df['bikes_available'] 
        
                for tr_idx, va_idx in tss.split(df):
                    tr_x, va_x = x.iloc[tr_idx], x.iloc[va_idx]
                    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
        
            elif i > 0:
                df = train[train['station_id'] == i]
        
                x = df.drop(['id', 'predict', 'date', 'events', 'bikes_available'], axis=1)
                y = df['bikes_available'] 
        
                for tr_idx, va_idx in tss.split(df):
                    tr_x_, va_x_ = x.iloc[tr_idx], x.iloc[va_idx]
                    tr_y_, va_y_ = y.iloc[tr_idx], y.iloc[va_idx]
                
                tr_x = pd.concat([tr_x,tr_x_])
                va_x = pd.concat([va_x,va_x_])
                tr_y = pd.concat([tr_y,tr_y_])
                va_y = pd.concat([va_y,va_y_])
        
        

        lgb_train = lgb.Dataset(tr_x,tr_y)
        lgb_eval = lgb.Dataset(va_x,va_y)
        
        params = {'objective': 'regression',
                'metric': 'rmse',
                'feature_pre_filter': False,
                'lambda_l1': 3.0219157234596604e-06,
                'lambda_l2': 2.6309832619595715e-05,
                'num_leaves': 137,
                'feature_fraction': 0.8,
                'bagging_fraction': 1.0,
                'bagging_freq': 0,
                'min_child_samples': 5,
                'num_iterations': 1000,
                'early_stopping_round': 100}
                

        model = lgb.train(params,lgb_train,valid_sets=[lgb_train,lgb_eval],
                        verbose_eval=100,num_boost_round=10000,
                        early_stopping_rounds=100)

        test_ = test.drop(['id', 'predict', 'date', 'events', 'bikes_available',], axis=1)
        prediction = model.predict(test_)
        
        test['bikes_available'] = prediction

        sub = test[['id','bikes_available']]

        ls_ = test['id'].tolist()

        for i in ls_:
            whole.loc[(whole['id'] == i), 'bikes_available'] = test.loc[(test['id'] == i), 'bikes_available'].values[0]
        

    elif i >= 1:
        print('############################################')
        print(str(i+1) + '/' + str(len(ls)))
        print('############################################')

        s = ls[i]

        train = whole[whole['date'] < s]
        train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
        test = whole[whole['date'] == s][whole['predict'] == 1]

        nan_ids = train[train['bikes_available'].isnull()]['id'].to_list()
        nan_ids[len(nan_ids):len(nan_ids)] = test[test['bikes_available'].isnull()]['id'].to_list()

        for i in range(len(nan_ids)):
            week_ago = whole[whole['id'] == nan_ids[i]]['date'] - timedelta(days=7)
            week_ago = week_ago.values[0]

            if (whole[whole['id'] == nan_ids[i]]['holiday'] == whole[whole['date'] == week_ago].iloc[0,9]).values[0]:
                week_ago = week_ago
            else:
                week_ago = whole[whole['id'] == nan_ids[i]]['date'] - timedelta(days=14)
                week_ago = week_ago.values[0]

            station_id = whole[whole['id'] == nan_ids[i]]['station_id']
            station_id = station_id.values[0]
            hour = whole[whole['id'] == nan_ids[i]]['hour']
            hour = hour.values[0]

            whole.loc[(whole['id'] == nan_ids[i]), 'bikes_available'] = whole[(whole['date'] == week_ago) & (whole['hour'] == hour) & (whole['station_id'] == station_id)]['bikes_available'].values[0]

        train = whole[whole['date'] < s]
        #train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
        test = whole[whole['date'] == s][whole['predict'] == 1]

        print('NaN_train ' + str(train['bikes_available'].isnull().sum()))
        print('NaN_test ' + str(test['bikes_available'].isnull().sum()))
        
        tss = TimeSeriesSplit(n_splits=2, test_size=744)

        for i in range(70):
            
            if i == 0:
                df = train[train['station_id'] == i]
        
                x = df.drop(['id', 'predict', 'date', 'events', 'bikes_available'], axis=1)
                y = df['bikes_available'] 
        
                for tr_idx, va_idx in tss.split(df):
                    tr_x, va_x = x.iloc[tr_idx], x.iloc[va_idx]
                    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
        
            elif i > 0:
                df = train[train['station_id'] == i]
        
                x = df.drop(['id', 'predict', 'date', 'events', 'bikes_available'], axis=1)
                y = df['bikes_available'] 
        
                for tr_idx, va_idx in tss.split(df):
                    tr_x_, va_x_ = x.iloc[tr_idx], x.iloc[va_idx]
                    tr_y_, va_y_ = y.iloc[tr_idx], y.iloc[va_idx]
                
                tr_x = pd.concat([tr_x,tr_x_])
                va_x = pd.concat([va_x,va_x_])
                tr_y = pd.concat([tr_y,tr_y_])
                va_y = pd.concat([va_y,va_y_])

            
                
        lgb_train=lgb.Dataset(tr_x,tr_y)
        lgb_eval=lgb.Dataset(va_x,va_y)

        params = {'objective': 'regression',
                'metric': 'rmse',
                'feature_pre_filter': False,
                'lambda_l1': 3.0219157234596604e-06,
                'lambda_l2': 2.6309832619595715e-05,
                'num_leaves': 137,
                'feature_fraction': 0.8,
                'bagging_fraction': 1.0,
                'bagging_freq': 0,
                'min_child_samples': 5,
                'num_iterations': 1000,
                'early_stopping_round': 100}

        model=lgb.train(params,lgb_train,valid_sets=[lgb_train,lgb_eval],
                        verbose_eval=100,num_boost_round=10000,
                        early_stopping_rounds=100)

        test_ = test.drop(['id', 'predict', 'date', 'events', 'bikes_available'], axis=1)
        prediction = model.predict(test_)
        
        test['bikes_available'] = prediction
        
        sub_ = test[['id','bikes_available']]
        sub = pd.concat([sub, sub_ ], ignore_index=True)

        ls_ = test['id'].tolist()

        for i in ls_:
            whole.loc[(whole['id'] == i), 'bikes_available'] = test.loc[(test['id'] == i), 'bikes_available'].values[0]

        

############################################
1/120
############################################


/tmp/ipykernel_284696/2876090964.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
/tmp/ipykernel_284696/2876090964.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = whole[whole['date'] == s][whole['predict'] == 1]
/tmp/ipykernel_284696/2876090964.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = whole[whole['date'] == s][whole['predict'] == 1]


NaN_train 0
NaN_test 0


/home/yumezu/anaconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/yumezu/anaconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/yumezu/anaconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1099
[LightGBM] [Info] Number of data points in the train set: 539424, number of used features: 38
[LightGBM] [Info] Start training from score 8.602239
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 2.51461	valid_1's rmse: 3.18134
Early stopping, best iteration is:
[37]	training's rmse: 2.70729	valid_1's rmse: 3.16479
############################################
2/120
############################################


/tmp/ipykernel_284696/2876090964.py:114: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = train.append(whole[whole['date'] == s][whole['hour'] == 0])
/tmp/ipykernel_284696/2876090964.py:115: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = whole[whole['date'] == s][whole['predict'] == 1]
/tmp/ipykernel_284696/2876090964.py:139: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = whole[whole['date'] == s][whole['predict'] == 1]


NaN_train 0
NaN_test 0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068385 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 544464, number of used features: 38
[LightGBM] [Info] Start training from score 8.602018
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 2.52864	valid_1's rmse: 3.12468
Early stopping, best iteration is:
[62]	training's rmse: 2.62704	valid_1's rmse: 3.12175
############################################
3/120
############################################
NaN_train 0
NaN_test 0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [I

# 特徴量重要度を保管する dataframe を用意
feature_importances = pd.DataFrame()

for fold, model in enumerate(models):

    tmp = pd.DataFrame()
    tmp['feature'] = model.feature_name()
    tmp['importance'] = model.feature_importance(importance_type='gain')
    tmp['fold'] = fold

    feature_importances = feature_importances.append(tmp)

# 各特徴量で集約して、重要度の平均を算出。上位50個だけ抜き出す
order = list(feature_importances.groupby("feature")["importance"].mean().sort_values(ascending=False).index)[:50]

# 可視化
plt.figure(figsize=(10, 10))
sns.barplot(x='importance', y='feature', data=feature_importances, order=order)
plt.title('LGBM importance')
plt.tight_layout()
plt.show()

In [ ]:
sub.to_csv('sub_lightgbm.csv', index=False, header=False)